In [ ]:
#sklearn实现-线性阈值单元（LTU）

import numpy as np
from sklearn.datasets import load_iris 
from sklearn.linear_model import Perceptron

iris = load_iris()
x = iris.data[:, (2, 3)] #花瓣长度，宽度
y = (iris.target == 0).astype(np.int)
per_clf = Perceptron(random_state = 42)
per_clf.fit(x, y)

y_pred = per_clf.predict([[2, 0.5]])

In [ ]:
y_pred
#系数
per_clf.coef_
#截距
per_clf.intercept_

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
a = -per_clf.coef_[0][0] / per_clf.coef_[0][1]
b = -per_clf.intercept_ / per_clf.coef_[0][1]

axes = [0, 5, 0, 2]

x0, x1 = np.meshgrid(
    np.linspace(axes[0], axes[1], 500).reshape(-1, 1),
    np.linspace(axes[2], axes[3], 200).reshape(-1, 1)
)

x_new = np.c_[x0.ravel(), x1.ravel()]
print("x_new num:", x_new.shape)
y_predict = per_clf.predict(x_new)
zz = y_predict.reshape(x0.shape)

plt.figure(figsize=(10, 8))
plt.plot(x[y==0, 0], x[y==0, 1], "bs", label="not iris setosa")
plt.plot(x[y==1, 0], x[y==1, 1], "yo", label="iris setosa")

plt.plot([axes[0], axes[1]], [a * axes[0] + b, a*axes[1] + b], "k-", linewidth=3)

from matplotlib.colors import ListedColormap

custom_cmap = ListedColormap(['#9898ff', '#fafab0'])

plt.contourf(x0, x1, zz, cmap=custom_cmap)
plt.xlabel("petal length", fontsize=14)
plt.ylabel("petal width", fontsize=14)
plt.legend(loc="lower right", fontsize=14)
#设置无效xlim和ylim无效，因为设置了plt.axis()
#plt.xlim((-5, 5))
#plt.ylim((-5, 5))
plt.axis(axes)

plt.show()

In [ ]:
print(a * axes[0] + b, a*axes[1] + b)

In [ ]:
#激活函数
def logit(z):
    return 1 / (1 + np.exp(-z))

def relu(z):
    return np.maximum(0, z)

def derivative(f, z, eps=0.0000001):
    return (f(z + eps) - f(z - eps)) / (2 * eps)

z = np.linspace(-5, 5, 200)

plt.figure(figsize=(11, 4))
plt.subplot(121)
plt.plot(z, np.sign(z), "r-", linewidth=2, label="step")
plt.plot(z, logit(z), "g--", linewidth=2, label="logit")
plt.plot(z, np.tanh(z), "b--", linewidth=2, label="tanh")
plt.plot(z, relu(z), "m--", linewidth=2, label="relu")
plt.grid(True)#网格线

plt.legend(loc="center right", fontsize=14)
plt.title("Activation functions", fontsize=14)
plt.axis([-5, 5, -1.2, 1.2])

plt.subplot(122)
plt.plot(z, derivative(np.sign, z), "r-", linewidth=2, label="Step")
plt.plot(0, 0, "ro", markersize=5)
plt.plot(0, 0, "rx", markersize=10)
plt.plot(z, derivative(logit, z), "g--", linewidth=2, label="Logit")
plt.plot(z, derivative(np.tanh, z), "b-", linewidth=2, label="Tanh")
plt.plot(z, derivative(relu, z), "m-.", linewidth=2, label="ReLU")
plt.grid(True)
#plt.legend(loc="center right", fontsize=14)
plt.title("Derivatives", fontsize=14)
plt.axis([-5, 5, -0.2, 1.2])

plt.show()

In [ ]:
#多层感知器（MLP）
#MLP 由一个（通过）输入层、一个或多个称为隐藏层的 LTU 组成，一个最终层 LTU 称为输出层
#以下两个隐藏层的 DNN（一个具有 300 个神经元，另一个具有 100 个神经元）和一个具有 10 个神经元的 SOFTMax 输出层进行分类
import tensorflow as tf

feature_columns = tf.contrib.learn.inferz_real_valued_columns_from_input(x_train)
dnn_clf = tf.contrib.learn.DNNClassifier(hidden_units=[300,100], 
                                         n_classes=10,
                                        feature_columns = feature_columns)

dnn_clf.fit(x=x_train, y=y_train, batch_size=50, steps=40000)

In [ ]:
#使用TensorFlow训练DNN

import tensorflow as tf
#指定输入和输出的数量，并设置每个层中隐藏的神经元数量
n_inputs = 28*28
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

x = tf.placeholder(tf.float32, shape=(None, n_inputs), name="x")
y = tf.placeholder(tf.int32, shape=(None), name="y")



In [ ]:
#创建神经元层
def neuron_layer(x, n_neurons, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(x.get_shape()[1])
        stddev = 2 / np.sqrt(n_inputs)
        init = tf.truncated_normal((n_inputs, n_neurons), stddev = stddev)
        w = tf.Variable(init, name="weigths")
        b = tf.Variable(tf.zeros([n_neurons]), name="biases")
        z = tf.matmul(x, w) + b
        
        if activation == 'relu':
            return tf.nn.relu(z)
        else:
            return z
        
        

In [ ]:
#创建深层神经网络
#第一个隐藏层以X为输入。 
#第二个将第一个隐藏层的输出作为其输入。 #
#最后，输出层将第二个隐藏层的输出作为其输入。

with tf.name_scope("dnn"):
    hidden1 = neuron_layer(x, n_hidden1, name='hidden1', activation=tf.nn.relu)
    hidden2 = neuron_layer(hidden1, n_hidden2, name="hidden2", activation=tf.nn.relu)
    logits = neuron_layer(hidden2, n_outputs, name="outputs")

In [ ]:
#定义损失函数
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

In [ ]:
#优化器
learning_rate = 0.01
with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)


In [ ]:
#评估模型
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [ ]:
#初始化所有节点
init = tf.global_variables_initializer()
#保存模型
saver = tf.train.Saver()

In [ ]:
#加载数据
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("/tmp/data/")

In [ ]:
#训练模型
n_epochs = 300
batch_size = 50

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            x_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={x:x_batch, y:y_batch})
        acc_train = accuracy.eval(feed_dict={x:x_batch, y:y_batch})
        acc_test = accuracy.eval(feed_dict={x: mnist.test.images, y: mnist.test.labels})
        print(epoch, "train accuracy:", acc_train, "test accuracy:", acc_test)
        
    sava_path = saver.save(sess, "./my_model_final.cpkt")

In [ ]:
#使用训练好的模型
with tf.Session() as sess:
    saver.restore(sess, "./my_model_final.ckpt")
    x_new_scaled = mnist.test.images[:20]
    z = logits.eval(feed_dict={x:x_new_scaled})
    y_pred = np.argmax(z, axis=1)